<b>Introduction/Business Problem:</b>
<br>Montreal is a city covering 431.5 km^2 (166.6 mi^2) with approx. 1.8M inhabitants residing in 21 boroughs.

The City of Saints is visited by over 2M people each year; making it Canada's most visited city. With so much to see and do, I'm going to need a cup of coffee!

With that in mind, I'm going to examine The City of 100 Steeples and see which neighborhoods will shelter and stimulate weary travelers.

<b>Data:</b> 
<br>I will be using zipcode for metropolitan Montreal that is available on wikipedia. I will also be using venue name, location, and category information on Foursquare to identify neighborhoods that have accomodations for travelers (hotels/hostels/inns/etc) and coffee shops.

In [44]:
#putting tools in the toolbox
import pandas as pd
import numpy as np
import requests as re
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request
import os
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
!pip -q install folium
import folium
print('Done')

Done


In [75]:
#linking our postal code and neighborhood data
zipcode_url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_H"
source=requests.get(list_url).text
#print(source)

In [61]:
#looking for patterns in the source code
soup=BeautifulSoup(source)
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: H - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"a9919fe9-87d1-4568-9755-faa3935e1b7d","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_H","wgTitle":"List of postal codes of Canada: H","wgCurRevisionId":961875995,"wgRevisionId":961875995,"wgArticleId":576883,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Quebec","Postal codes in Canada","Dollar

In [96]:
#seperating postal code and neighborhood data from the rest of the page
#codes=soup.find_all('td',attrs={'valign':'top'})
#for code in codes:
#    print(code)
code_body=soup.find('tbody')
#code_body

In [105]:
#making a list of postal codes
postal_codes=code_body.find_all('b')
#postal_codes
code_list=[]
for p in postal_codes[0:]:
    result=p.text.strip()
    code_list.append(result)
print(code_list)

['H0A,', 'H1A', 'H2A', 'H3A', 'H4A', 'H5A', 'H7A', 'H8A', 'H9A', 'H0B', 'H1B', 'H2B', 'H3B', 'H4B', 'H5B', 'H7B', 'H8B', 'H9B', 'H0C', 'H1C', 'H2C', 'H3C', 'H4C', 'H5C', 'H7C', 'H8C', 'H9C', 'H0E', 'H1E', 'H2E', 'H3E', 'H4E', 'H5E', 'H7E', 'H8E', 'H9E', 'H0G', 'H1G', 'H2G', 'H3G', 'H4G', 'H5G', 'H7G', 'H8G', 'H9G', 'H0H', 'Reserved', 'H1H', 'H2H', 'H3H', 'H4H', 'H5H', 'H7H', 'H8H', 'H9H', 'H0J', 'H1J', 'H2J', 'H3J', 'H4J', 'H5J', 'H7J', 'H8J', 'H9J', 'H0K', 'H1K', 'H2K', 'H3K', 'H4K', 'H5K', 'H7K', 'H8K', 'H9K', 'H0L', 'H1L', 'H2L', 'H3L', 'H4L', 'H5L', 'H7L', 'H8L', 'H9L', 'H0M', 'H1M', 'H2M', 'H3M', 'H4M', 'H5M', 'H7M', 'H8M', 'H9M', 'H0N', 'H1N', 'H2N', 'H3N', 'H4N', 'H5N', 'H7N', 'H8N', 'H9N', 'H0P', 'H1P', 'H2P', 'H3P', 'H4P', 'H5P', 'H7P', 'H8P', 'H9P', 'H0R', 'H1R', 'H2R', 'H3R', 'H4R', 'H5R', 'H7R', 'H8R', 'H9R', 'H0S', 'H1S', 'H2S', 'H3S', 'H4S', 'H5S', 'H7S', 'H8S', 'H9S', 'H0T', 'H1T', 'H2T', 'H3T', 'H4T', 'H5T', 'H7T', 'H8T', 'H9T', 'H0V', 'H1V', 'H2V', 'H3V', 'H4V', 'H5V',

In [106]:
len(code_list)

181

In [113]:
#making a list of neighborhoods
neighborhoods=code_body.find_all('span')
#print(neighborhoods)
ngh_list=[]
for n in neighborhoods[0:]:
    ngh_results=n.text.strip()
    ngh_list.append(ngh_results)
print(ngh_list)

['Not assigned', 'Pointe-aux-Trembles', 'Saint-Michel,East', 'Downtown Montreal North(McGill University)', 'Notre-Dame-de-GrâceNortheast', 'Place Bonaventure', 'Duvernay-Est', 'Not assigned', 'Dollard-des-OrmeauxNorthwest', 'Not assigned', 'Montreal East', 'AhuntsicNorth', 'Downtown MontrealEast', 'Notre-Dame-de-GrâceSouthwest', 'Place Desjardins', 'Saint-François', 'Not assigned', 'Dollard-des-OrmeauxEast', 'Not assigned', 'Rivière-des-PrairiesNortheast', 'AhuntsicCentral', 'Griffintown(Includes Île Notre-Dame & Île Sainte-Hélène)(Université de Montréal)', 'Saint-Henri', 'Not assigned', 'Saint-Vincent-de-Paul', 'Not assigned', "L'Île-BizardNortheast", 'Not assigned', 'Rivière-des-PrairiesSouthwest', 'VillerayNortheast', "L'Île-Des-Soeurs", 'Ville Émard', 'Not assigned', 'Duvernay', 'Not assigned', "L'Île-BizardSouthwest", 'Not assigned', 'Montréal-NordNorth', 'Petite-PatrieNortheast', 'Downtown MontrealSoutheast (Concordia University)', 'VerdunNorth', 'Not assigned', 'Pont-Viau', 'Not

In [114]:
len(ngh_list)

181

In [152]:
#create data frame with postal codes and neighborhoods
mtl_ngh=pd.DataFrame({'Postalcode':code_list,'Neighborhood':ngh_list})
mtl_ngh.head()

,Postalcode,Neighborhood
0,"H0A,",Not assigned
1,H1A,Pointe-aux-Trembles
2,H2A,"Saint-Michel,East"
3,H3A,Downtown Montreal North(McGill University)
4,H4A,Notre-Dame-de-GrâceNortheast


In [156]:
#remove postal codes that are not assigned
mtl_ngh=mtl_ngh[mtl_ngh['Neighborhood']!='Not assigned']
print(mtl_ngh.head())
len(mtl_ngh)

  Postalcode                                Neighborhood
1        H1A                         Pointe-aux-Trembles
2        H2A                           Saint-Michel,East
3        H3A  Downtown Montreal North(McGill University)
4        H4A                Notre-Dame-de-GrâceNortheast
5        H5A                           Place Bonaventure


124

In [157]:
mtl_temp=mtl_ngh.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
mtl_temp=mtl_temp.reset_index(drop=False)
mtl_temp.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)
mtl_temp=mtl_temp[mtl_temp['Neighborhood_joined']!='Reserved0H0: Santa Claus']
print(mtl_temp.head())
len(mtl_temp)

  Postalcode Neighborhood_joined
1        H0J           AnjouWest
2        H0K           AnjouEast
3        H0L        MercierNorth
4        H0M         MercierWest
5        H0N    MercierSoutheast


123

In [163]:
mtl_merge=pd.merge(mtl_ngh,mtl_temp,on='Postalcode')
mtl_merge.drop(['Neighborhood'],axis=1,inplace=True)
mtl_merge.drop_duplicates(inplace=True)
mtl_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
mtl_merge.head()

,Postalcode,Neighborhood
0,H1A,Pointe-aux-Trembles
1,H2A,"Saint-Michel,East"
2,H3A,Downtown Montreal North(McGill University)
3,H4A,Notre-Dame-de-GrâceNortheast
4,H5A,Place Bonaventure


In [166]:
def get_mtlgeo(postal_code):
    lat_lng_coord is None
    while(lat_lng_coord is None):
        m = geocoder.google('{}, Montreal, Quebec'.format(postal_code))
        lat_lng_coord=m.latlng
    mtl_lat=lat_lng_coord[0]
    mtl_lng=lat_lng_coord[1]
    return mtl_lat,mtl_lng
mtl_geo=pd.read_csv('http://cocl.us/Geospatial_data')
mtl_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
